# Skip-gram Word2Vec

In this notebook, I'll lead you through using PyTorch to implement the [Word2Vec algorithm](https://en.wikipedia.org/wiki/Word2vec) using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of Word2Vec from Chris McCormick 
* [First Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [Neural Information Processing Systems, paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for Word2Vec also from Mikolov et al.

---
## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So, the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

<img src='assets/one_hot_encoding.png' width=50%>

To solve this problem and greatly increase the efficiency of our networks, we use what are called **embeddings**. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

<img src='assets/lookup_matrix.png' width=50%>

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=50%>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.

---
## Word2Vec

The Word2Vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words.

<img src="assets/context_drink.png" width=40%>

Words that show up in similar **contexts**, such as "coffee", "tea", and "water" will have vectors near each other. Different words will be further away from one another, and relationships can be represented by distance in vector space.

<img src="assets/vector_distance.png" width=40%>


There are two architectures for implementing Word2Vec:
>* CBOW (Continuous Bag-Of-Words) and 
* Skip-gram

<img src="assets/word2vec_architectures.png" width=60%>

In this implementation, we'll be using the **skip-gram architecture** because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

---
## Loading Data

Next, we'll ask you to load in data and place it in the `data` directory

1. Load the [text8 dataset](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip); a file of cleaned up *Wikipedia article text* from Matt Mahoney. 
2. Place that data in the `data` folder in the home directory.
3. Then you can extract it and delete the archive, zip file to save storage space.

After following these steps, you should have one file in your data directory: `data/text8`.

In [1]:
# read in the extracted text file      
with open('data/text8') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Pre-processing

Here I'm fixing up the text to make training easier. This comes from the `utils.py` file. The `preprocess` function does a few things:
>* It converts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. 
* It removes all words that show up five or *fewer* times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. 
* It returns a list of words in the text.

This may take a few seconds to run, since our text file is quite large. If you want to write your own functions for this stuff, go for it!

In [2]:
import utils

# get list of words
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'of', 'the', 'french', 'revolution', 'whilst', 'the']


In [3]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 16616688
Unique words: 53721


### Dictionaries

Next, I'm creating two dictionaries to convert words to integers and back again (integers to words). This is again done with a function in the `utils.py` file. `create_lookup_tables` takes in a list of words in a text and returns two dictionaries.
>* The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1, and so on. 

Once we have our dictionaries, the words are converted to integers and stored in the list `int_words`.

In [4]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 1, 0, 150, 854, 3580, 0]


## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

I'm going to leave this up to you as an exercise. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [5]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5
word_counts = Counter(int_words)
print(list(word_counts.items())[0])  # dictionary of int_words, how many times they appear

total_number_of_words = len(int_words)
frequencies = {word: count / total_number_of_words for word, count in word_counts.items()}


# discard some frequent words, according to the subsampling equation
# create a new list of words for training
def should_be_discarded(word):
    ''' word is an integer'''
    assert word in word_counts
    discard_probability = 1 - np.sqrt(threshold / frequencies[word])
    return random.random() < discard_probability

train_words = [word for word in int_words if not should_be_discarded(word)]

print(train_words[:30])

(5233, 303)
[155, 10571, 133, 27349, 15067, 854, 58, 10712, 1423, 2757, 686, 7088, 1052, 248, 44611, 5233, 2621, 8983, 4147, 6437, 4186, 5233, 1818, 6753, 7573, 1774, 566, 93, 11064, 7088]


In [6]:
print([int_to_vocab[word] for word in train_words[:30]])

['against', 'radicals', 'including', 'diggers', 'sans', 'revolution', 'used', 'pejorative', 'positive', 'label', 'defined', 'anarchists', 'derived', 'without', 'archons', 'anarchism', 'rulers', 'unnecessary', 'abolished', 'differing', 'interpretations', 'anarchism', 'movements', 'elimination', 'authoritarian', 'institutions', 'particularly', 'state', 'anarchy', 'anarchists']


## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to define a surrounding _context_ and grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $[ 1: C ]$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you chose a random number of words to from the window.

Say, we have an input and we're interested in the idx=2 token, `741`: 
```
[5233, 58, 741, 10571, 27349, 0, 15067, 58112, 3580, 58, 10712]
```

For `R=2`, `get_target` should return a list of four values:
```
[5233, 58, 10571, 27349]
```

In [7]:
def get_target(words, index, window_size=5):
    ''' Get a list of words in a window around an index. '''
    r = np.random.randint(1, window_size+1)
    if r > index:
        return words[:index] + words[index + 1: index + r + 1]
    elif r > len(words) - index:
        return words[index - r:index] + words[index + 1:]
    else:
        return words[index - r:index] + words[index + 1: index + r + 1]


### Generating Batches 

Here's a generator function that returns batches of input and target data for our model, using the `get_target` function from above. The idea is that it grabs `batch_size` words from a words list. Then for each of those batches, it gets the target words in a window.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

In [9]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 0, 1, 1, 1, 2, 2, 3, 3]
y
 [1, 2, 3, 0, 2, 3, 1, 3, 1, 2]


## Building the graph

Below is an approximate diagram of the general structure of our network.
<img src="assets/skip_gram_net_arch.png" width=60%>

>* The input words are passed in as batches of input word tokens. 
* This will go into a hidden layer of linear units (our embedding layer). 
* Then, finally into a softmax output layer. 

We'll use the softmax layer to make a prediction about the context words by sampling, as usual.

The idea here is to train the embedding layer weight matrix to find efficient representations for our words. We can discard the softmax layer because we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in _other_ networks we build using this dataset.

---
## Validation

Here, I'm creating a function that will help us observe our model as it learns. We're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them using the cosine similarity: 

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


We can encode the validation words as vectors $\vec{a}$ using the embedding table, then calculate the similarity with each word vector $\vec{b}$ in the embedding table. With the similarities, we can print out the validation words and words in our embedding table semantically similar to those words. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [10]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## SkipGram model

Define and train the SkipGram model. 
> You'll need to define an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) and a final, softmax output layer.

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

In [11]:
import torch
from torch import nn
import torch.optim as optim

In [14]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        # complete this SkipGram model
        self.embed = nn.Embedding(num_embeddings=n_vocab, embedding_dim=n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        
        # define the forward behavior
        x = self.embed(x)
        x = self.output(x)
        x = self.log_softmax(x)
        return x

### Training

Below is our training loop, and I recommend that you train on GPU, if available.

**Note that, because we applied a softmax function to our model output, we are using NLLLoss** as opposed to cross entropy. This is because Softmax  in combination with NLLLoss = CrossEntropy loss .

In [ ]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim=300 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 100
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(train_words, 512):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

which | fortified, ehr, molarity, moisture, semon
more | vanadium, dollars, helier, describe, happiness
if | kidnappers, hestia, ht, encephalopathy, ancestries
however | racing, beatty, huldrych, dept, prairies
or | euphoria, linkin, oversaw, grok, greek
only | spiritualism, microcomputers, congenial, tubas, filtering
so | migrant, plummeted, knobs, sadozai, smirnov
that | bosco, sident, mandeville, cronquist, ocular
placed | rapids, embryo, medieval, volhynia, limelight
something | reportedly, rivi, euthanasia, starboard, thermotropic
troops | combines, precambrian, eesti, catchers, distillery
file | sings, editorial, khmelnytsky, carbonation, timorese
experience | menagerie, bleak, dtmf, nut, won
engine | affixes, ilya, senex, ministers, shower
bible | tactically, fighting, infertile, chinon, centum
shows | distract, liang, narcolepsy, zagreb, codified
...
see | rigged, superficially, vowel, soldiery, display
with | accent, corroborating, fibre, penderecki, soc
over | untamed, spawni

to | milah, clark, versa, circumnavigation, breaking
over | untamed, spawning, leakey, aftermath, holden
may | transformation, lawyer, assumed, ges, subversion
is | wilfrid, amiga, opponents, tyler, hunchback
often | drive, billiard, imprisonment, wail, digitized
where | hoods, machinations, significant, salvador, scuttled
six | michelson, pepe, acquired, citadels, enforces
its | mezzo, ovum, adjective, reefs, leftmost
liberal | druze, reluctantly, governors, quality, available
arts | xinhua, pennyroyal, consults, silvery, friendly
shows | liang, zagreb, distract, narcolepsy, codified
existence | schuman, interwiki, willamette, soundhole, euskadi
writers | migrating, irritability, jessica, cady, perishable
derived | elimination, outrun, kits, monism, bei
proposed | articulatory, prophesy, toolbox, agnostic, psychosomatic
mathematics | sybil, bax, magenta, mellow, icon
...
war | herndon, daoism, social, alison, complements
history | wells, modernize, outlines, thurston, damian
or | euph

where | machinations, hoods, salvador, significant, gahan
so | migrant, thracian, maariv, dog, conspired
with | soc, hospitals, accent, qdos, corroborating
by | regan, mattingly, permafrost, floods, speaker
nine | archbishops, haworth, saeed, carrero, whigs
of | cartman, curiously, housing, timbers, plummet
or | linkin, euphoria, transept, oni, trnc
system | relation, jai, cdb, brindle, dmu
consists | embellished, higgs, withstand, freefall, revived
cost | attacking, cisc, finale, havoc, uno
paris | chd, unger, evan, unfinished, antinomy
resources | hominoid, allude, freeman, smear, sys
older | madeleine, integrative, calibration, icosidodecahedron, recipe
ice | aberystwyth, temmu, lsch, septum, sketched
scale | swears, unwitting, sar, untamed, actaeon
magazine | eliminates, leakey, consumer, ollie, wronged
...
was | refining, eduke, southey, inhumane, gainesville
that | mandeville, examinations, quranic, ocular, bosco
can | impression, neoplatonist, vowels, very, pea
by | regan, matti

no | abrahams, death, majority, punctured, abdullah
seven | canseco, itasca, singer, loretta, sandpaper
they | altering, parametric, especially, absorptive, mukhabarat
known | aortic, ratifies, sidi, yokosuka, ecce
american | craig, deposed, replicant, tallest, cartier
most | pasadena, nabataean, shalom, hearts, quantify
these | conveying, decapitated, louth, bona, scraped
see | confucian, braun, doubling, boardgamegeek, superficially
additional | onions, sales, refreshed, gotha, rard
operating | outputting, lavishly, ala, piet, axles
road | wharton, affonso, hyenas, pei, outside
pre | markt, morgue, sampo, pneumonic, arguing
pressure | jis, stile, systemic, dwelt, lux
resources | hominoid, freeman, allude, smear, exerts
prince | codebase, nazism, lynching, treads, defensible
animals | marriott, mahathir, render, dominant, literal
...
five | basra, forecasts, liddell, edison, zero
see | confucian, braun, doubling, boardgamegeek, superficially
war | withdrew, herndon, daoism, social, de

its | adjective, mezzo, reefs, leftmost, ovum
if | ht, hestia, kidnappers, stroke, feces
he | tabari, yogi, moqed, attend, merits
d | astro, dp, video, bond, blume
often | drive, imprisonment, pursuits, digitized, otherworldly
a | wisest, guinness, aleutians, essentially, slugger
from | clears, perpetuating, noh, diameter, resolutely
for | footnote, promises, convergent, implementation, irl
lived | corse, winch, hotly, clinics, victors
behind | try, collier, buddhism, rivi, immediately
dr | nonaligned, sailing, thrusting, theories, interviews
derived | outrun, kits, elimination, theory, histological
articles | adamant, gramophone, edgerton, nuremberg, interdependence
alternative | alienation, ernest, widow, encampments, herring
applications | pisa, gladiators, functional, controller, vladimir
notes | doubly, ezek, abbe, nationalisation, ontogeny
...
often | drive, imprisonment, pursuits, digitized, otherworldly
such | mcfadden, consultations, maguey, cellular, hattie
four | three, rolf

...

up | out, back, get, they, were
use | used, such, common, standard, or
that | all, be, thus, say, which
it | is, so, be, exist, if
over | zero, four, years, from, six
which | be, form, is, a, called
war | army, troops, allied, soldiers, forces
eight | nine, one, four, six, three
behind | back, ahead, lateral, yankees, front
existence | universe, exist, ontological, belief, things
rise | decline, increased, rising, peaked, grew
road | highway, west, rail, roads, highways
file | files, user, data, format, formats
smith | sidney, roberts, young, miller, jr
know | we, you, knows, thing, nobody
woman | children, she, women, husband, birth
...
are | is, have, usually, or, these
four | two, three, five, seven, zero
who | him, they, was, his, their
zero | two, four, five, three, six
most | are, many, some, other, these
be | can, this, which, should, to
which | be, is, a, form, that
united | states, kingdom, countries, america, panama
marriage | marriages, marry, spouse, wife, her
road | highwa

their | they, them, themselves, tend, these
was | had, after, early, s, during
people | deaths, births, politicians, activists, americans
nine | one, eight, three, zero, two
seven | one, eight, five, four, six
war | troops, soviet, forces, soldiers, army
many | some, well, more, several, most
may | or, are, occur, usually, sometimes
smith | adam, jefferson, keith, joseph, gordon
animals | animal, prey, mammals, humans, predators
quite | very, than, like, rounded, always
engineering | engineers, technology, engineer, education, biomedical
award | awards, academy, oscars, nominations, best
report | reports, commission, review, january, notified
numerous | become, early, many, and, various
universe | cosmology, bang, galaxies, universes, cosmic
...
states | united, treaties, treaty, establishment, u
s | his, was, of, and, nine
d | b, one, seven, statesman, writer
three | two, four, one, nine, six
people | deaths, births, politicians, americans, activists
th | century, rd, nd, st, centurie

it | if, this, not, so, they
see | article, links, also, external, history
from | of, the, to, in, at
known | important, named, referred, called, originated
by | the, and, in, of, s
in | the, also, by, of, a
or | is, usually, are, may, other
are | all, other, is, these, have
centre | centres, largest, situated, city, gallery
shows | show, showing, theatrical, michaels, guest
paris | la, de, le, france, mie
derived | word, meaning, name, suffix, derives
professional | juris, engineering, bachelor, professionals, amateur
heavy | fog, storms, humidity, metal, low
prince | emperor, queen, reigned, empress, pretender
creation | created, conceived, themes, inspired, god
...
had | were, remained, meantime, was, afterwards
it | if, this, so, they, is
first | s, also, was, a, final
one | four, three, seven, two, nine
three | four, one, two, seven, zero
most | many, in, some, such, modern
he | his, him, himself, wrote, she
for | a, of, tools, allows, which
running | ran, run, weicker, grub, fini

but | to, however, had, they, the
one | eight, three, two, zero, seven
by | of, the, in, as, to
zero | three, five, four, two, one
can | be, requires, or, use, usually
this | the, not, as, thus, that
only | the, except, any, be, but
six | seven, five, four, eight, one
bbc | programmes, news, television, links, profile
rise | decline, increased, peaked, europe, fall
egypt | egyptian, israel, arab, pharaoh, cairo
shows | show, theatrical, guest, television, wow
square | approximately, sq, coastline, cubic, km
proposed | proposal, approved, agreement, theories, signatory
brother | son, father, daughter, sisters, mother
recorded | record, kush, khz, recordings, records
...
s | of, and, in, the, secretary
his | he, him, himself, father, friend
th | century, rd, nd, st, lost
often | usually, such, some, or, are
than | less, fewer, large, are, higher
would | could, be, to, when, even
between | differences, both, to, neutral, assert
was | until, first, in, later, became
account | estimates, ac

four | five, three, two, six, zero
was | a, the, in, had, after
about | zero, approximately, five, there, three
where | left, moved, apartments, separated, near
people | living, americans, demographics, citizens, activists
at | near, years, home, zero, in
all | are, these, not, other, there
they | themselves, their, not, do, these
alternative | such, based, used, type, abbreviations
troops | army, war, forces, soldiers, battle
paris | france, la, de, mie, acad
pope | papal, antipope, papacy, church, gregory
joseph | benjamin, biography, james, sympathetic, louis
frac | cdot, equation, theta, mathbf, cos
existence | disembodied, belief, demons, universe, philosophers
writers | novelists, authors, fiction, poets, births
...
d | b, composer, seven, eight, writer
during | after, the, was, period, continued
see | list, external, links, disambiguation, article
for | a, and, as, including, in
world | war, international, of, nine, ii
three | two, four, five, six, one
is | are, or, refers, also

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
# getting embeddings from the embedding layer of our model, by name
embeddings = model.embed.weight.to('cpu').data.numpy()

In [ ]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)